In [22]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('basics').getOrCreate()

In [23]:
# Importing data which has a header. Schema is automatically configured.
sales_data = spark.read.csv('rollingsales_nyc2019.csv', header=True, inferSchema=True)

# Let's see the data. You'll notice nulls.
sales_data.limit(5).toPandas()

,ID,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,1,Manhattan,ALPHABET CITY,01 ONE FAMILY DWELLINGS,1,400,19,None,A4,526 EAST 5TH STREET,...,1,0,1,1883,5200,1900,1,A4,6100000,3/12/2018
1,2,Manhattan,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,None,B9,166 AVENUE A,...,2,0,2,1510,4520,1900,1,B9,0,29/11/2018
2,3,Manhattan,ALPHABET CITY,02 TWO FAMILY DWELLINGS,1,404,1,None,B9,166 AVENUE A,...,2,0,2,1510,4520,1900,1,B9,0,29/11/2018
3,4,Manhattan,ALPHABET CITY,03 THREE FAMILY DWELLINGS,1,377,56,None,C0,263 EAST 7TH STREET,...,3,0,3,2430,3600,1899,1,C0,6300000,30/04/2019
4,5,Manhattan,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,373,19,None,C7,332 EAST 4TH STREET,...,28,2,30,4651,17478,1920,2,C7,14000000,9/01/2019


## Data Understanding

In [24]:
sales_data.count()

79621

In [25]:
# For type, we can use print schema. 
# But wait! What if you want to change the format of the data? Maybe change age to an integer instead of long?
sales_data.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- BOROUGH: string (nullable = true)
 |-- NEIGHBORHOOD: string (nullable = true)
 |-- BUILDING CLASS CATEGORY: string (nullable = true)
 |-- TAX CLASS AT PRESENT: string (nullable = true)
 |-- BLOCK: integer (nullable = true)
 |-- LOT: integer (nullable = true)
 |-- EASE-MENT: string (nullable = true)
 |-- BUILDING CLASS AT PRESENT: string (nullable = true)
 |-- ADDRESS: string (nullable = true)
 |-- APARTMENT NUMBER: string (nullable = true)
 |-- ZIP CODE: integer (nullable = true)
 |-- RESIDENTIAL UNITS: integer (nullable = true)
 |-- COMMERCIAL UNITS: integer (nullable = true)
 |-- TOTAL UNITS: integer (nullable = true)
 |-- LAND SQUARE FEET: integer (nullable = true)
 |-- GROSS SQUARE FEET: string (nullable = true)
 |-- YEAR BUILT: integer (nullable = true)
 |-- TAX CLASS AT TIME OF SALE: integer (nullable = true)
 |-- BUILDING CLASS AT TIME OF SALE: string (nullable = true)
 |--  SALE PRICE : long (nullable = true)
 |-- SALE DATE: string (

In [26]:
# We can use the describe method get some general statistics on our data too. Remember to show the DataFrame!
# But what about data type?
sales_data.describe().toPandas()

,summary,ID,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,count,79621,79621,79621,79621,79621,79621,79621,0,79621,...,44540,44540,44540,46389,62113,79621,79621,79621,79621,79621
1,mean,39811.0,None,None,None,1.6196742201784033,4401.547581668153,354.493940040944,None,None,...,2.83926807364167,0.32718904355635386,3.166457117198024,3914.4696371984737,3760.974449381762,1807.3508998882205,1.624031348513583,None,1271078.415091496,None
2,stddev,22984.747228687695,None,None,None,0.8279772307173136,3665.2890711503296,636.3318060565701,None,None,...,13.102490530106984,6.935774027550197,14.92577704896296,15748.16853107207,24267.49379039857,508.7754976148334,0.8025203518095368,None,1.2225343545780145E7,None
3,min,1,Bronx,1 BEACH,01 ONE FAMILY DWELLINGS,1,1,1,None,A0,...,0,0,0,0,########,0,1,A0,0,1-Apr
4,max,79621,Staten Island,WYCKOFF HEIGHTS,49 CONDO WAREHOUSES/FACTORY/INDUS,4,16350,9057,None,Z9,...,1327,1132,1348,1845000,999,2019,4,Z9,2155000000,9/12/2018


## Data Preparation

In [27]:
#Rename Column Header
for col in sales_data.columns:
    sales_data = sales_data.withColumnRenamed(col,col.strip().lower().replace(" ","_"))
sales_data.columns

['id',
 'borough',
 'neighborhood',
 'building_class_category',
 'tax_class_at_present',
 'block',
 'lot',
 'ease-ment',
 'building_class_at_present',
 'address',
 'apartment_number',
 'zip_code',
 'residential_units',
 'commercial_units',
 'total_units',
 'land_square_feet',
 'gross_square_feet',
 'year_built',
 'tax_class_at_time_of_sale',
 'building_class_at_time_of_sale',
 'sale_price',
 'sale_date']

In [28]:
#Select relevant data
sales_data = sales_data.select([c for c in sales_data.columns if c not in ["id","ease-ment", "apartment_number", "residential_units", "commercial_units", "total_units", "land_square_feet", "address","sale_date","neighborhood","zip_code","block","lot"]])
sales_data.columns

['borough',
 'building_class_category',
 'tax_class_at_present',
 'building_class_at_present',
 'gross_square_feet',
 'year_built',
 'tax_class_at_time_of_sale',
 'building_class_at_time_of_sale',
 'sale_price']

In [29]:
#Cleaning data with abnormal values
sales_data = sales_data[sales_data.sale_price > 100]
sales_data = sales_data[sales_data.year_built > 1000]
sales_data.describe().toPandas()

,summary,borough,building_class_category,tax_class_at_present,building_class_at_present,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price
0,count,51294,51294,51294,51294,37470,51294,51294,51294,51294
1,mean,None,None,1.59226184207755,None,3496.415318921804,1951.2193044020744,1.6060552891176356,None,1829846.5367099466
2,stddev,None,None,0.7278410983232105,None,23262.214397671956,34.8796477611941,0.71153272194856,None,1.5004439872354893E7
3,min,Bronx,01 ONE FAMILY DWELLINGS,1,A0,0,1030,1,A0,125
4,max,Staten Island,49 CONDO WAREHOUSES/FACTORY/INDUS,4,Z9,999,2019,4,Z9,2155000000


In [30]:
#nConstructing new column

import pyspark.sql.functions as F

sales_data = sales_data.withColumn(
    'price_category',
    F.when(F.col('sale_price').between(0, 250000), 'Low')\
    .when(F.col('sale_price').between(250001,500000), 'Middle-Low')\
    .when(F.col('sale_price').between(500001,750000), 'Middle-High')\
    .otherwise('High')
)
sales_data.limit(5).toPandas()

,borough,building_class_category,tax_class_at_present,building_class_at_present,gross_square_feet,year_built,tax_class_at_time_of_sale,building_class_at_time_of_sale,sale_price,price_category
0,Manhattan,01 ONE FAMILY DWELLINGS,1,A4,5200,1900,1,A4,6100000,High
1,Manhattan,03 THREE FAMILY DWELLINGS,1,C0,3600,1899,1,C0,6300000,High
2,Manhattan,07 RENTALS - WALKUP APARTMENTS,2,C7,17478,1920,2,C7,14000000,High
3,Manhattan,07 RENTALS - WALKUP APARTMENTS,2A,C2,6294,1900,2,C2,872500,High
4,Manhattan,07 RENTALS - WALKUP APARTMENTS,2B,C7,14347,1920,2,C7,1550000,High


In [38]:
#Select relevant data
sales_manhattan = sales_data.filter(sales_data['borough'] == "Manhattan")
sales_bronx = sales_data.filter(sales_data['borough'] == "Bronx")
sales_queens = sales_data.filter(sales_data['borough'] == "Queens")
sales_brooklyn = sales_data.filter(sales_data['borough'] == "Brooklyn")
sales_staten_island = sales_data.filter(sales_data['borough'] == "Staten Island")

In [42]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

sales_data = unionAll(sales_manhattan, sales_bronx, sales_queens, sales_brooklyn, sales_staten_island)

## Data Transformation

In [50]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder

indexer = StringIndexer(inputCol="borough", outputCol="borough_index")
inputs = [indexer.getOutputCol(), "category2"]

AttributeError: 'StringIndexer' object has no attribute 'show'